In [0]:
VOLUME = '/Volumes/hub/default/files'

In [0]:

path = f"{VOLUME}/excel/sales_single_sheet.xlsx"

df = (spark.read
      .option("headerRows", 1)
      .excel(path))

display(df)

In [0]:
path = f"{VOLUME}/excel/inventory_multi_sheet.xlsx"

sheets = (spark.read
          .format("excel")
          .option("operation", "listSheets")
          .load(path))

display(sheets)


In [0]:
path = f"{VOLUME}/excel/range_starts_my_table.xlsx"

df_range = (spark.read
            .format("excel")
            .option("headerRows", 1)
            .load(path))

display(df_range)

In [0]:
path = f"{VOLUME}/excel/excel_write"

(
    df_range.write.format("excel")
    .mode("overwrite")
    .save(path)
)

In [0]:
%sql
SELECT * FROM read_files(
  "/Volumes/hub/default/files/excel/range_starts_my_table.xlsx",
  format => 'excel',
  schemaEvolutionMode => 'none',
  recordName => 'MySheet',
  headerRows => 1
);

In [0]:
path = f"{VOLUME}/excel_stream/files"
schema_dir = f"{VOLUME}/excel_stream/stream_schema"
checkpoint = f"{VOLUME}/excel_stream/stream_checkpoint"

df_stream = (spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "excel")
  .option("headerRows", 1)
  .option("cloudFiles.inferColumnTypes", True)
  .option("cloudFiles.schemaLocation", schema_dir)
  .option("cloudFiles.schemaEvolutionMode", "none")  # required for Excel streaming
  .load(path)
)

target = "hub.default.excel_stream_bronze"

(
  df_stream.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint)
    .trigger(availableNow=True)
    .toTable(target)
)


In [0]:
display(spark.table(target))